In [1]:
import os
import mindspore
from mindnlp.transformers import AutoModelForSeq2SeqLM
from mindnlp.peft import get_peft_config, get_peft_model, get_peft_model_state_dict, LoraConfig, TaskType
from mindnlp.dataset import load_dataset
from mindnlp.core import ops

from mindnlp.transformers import AutoTokenizer
from mindnlp.common.optimization import get_linear_schedule_with_warmup
from tqdm import tqdm

model_name_or_path = "bigscience/mt0-large"
tokenizer_name_or_path = "bigscience/mt0-large"

checkpoint_name = "financial_sentiment_analysis_lora_v1.ckpt"
max_length = 128
lr = 1e-3
num_epochs = 3
batch_size = 8

[WARNING] ME(4027200:281473120071712,MainProcess):2024-10-21-16:48:33.250.525 [mindspore/run_check/_check_version.py:357] MindSpore version 2.3.1 and Ascend AI software package (Ascend Data Center Solution)version 7.5 does not match, the version of software package expect one of ['7.2', '7.3']. Please refer to the match info on: https://www.mindspore.cn/install
/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <c

In [2]:
# creating model
peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

MT5ForConditionalGeneration has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`.`PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB
trainable params: 2,359,296 || all params: 1,231,940,608 || trainable%: 0.19151053100118282


In [3]:
mindspore.dataset.config.set_seed(123)
# loading dataset
dataset = load_dataset("financial_phrasebank", "sentences_allagree")

Using the latest cached version of the module from /home/lvyufeng/.cache/huggingface/modules/datasets_modules/datasets/financial_phrasebank/dcba1275938e14aa0a7f6ba18fef92e9e957c0e5436dc58ad71c41c2906eb4f2 (last modified on Tue Aug 13 21:20:09 2024) since it couldn't be found locally at financial_phrasebank, or remotely on the Hugging Face Hub.


In [4]:
classes = dataset.source.ds.features["label"].names
classes

['negative', 'neutral', 'positive']

In [5]:
train_dataset, validation_dataset = dataset.shuffle(64).split([0.9, 0.1])

[WARNING] ME(4027200:281473120071712,MainProcess):2024-10-21-16:49:36.547.450 [mindspore/dataset/engine/datasets.py:1217] Dataset is shuffled before split.


In [6]:
def add_text_label(sentence, label):
    return sentence, label, classes[label.item()]

train_dataset = train_dataset.map(add_text_label, ['sentence', 'label'], ['sentence', 'label', 'text_label'])
validation_dataset = validation_dataset.map(add_text_label, ['sentence', 'label'], ['sentence', 'label', 'text_label'])

In [7]:
next(train_dataset.create_dict_iterator())

{'sentence': Tensor(shape=[], dtype=String, value= 'The gross area of the Innova 2 project will be about 10,000 sq m ( 107,600 sq ft ) .'),
 'label': Tensor(shape=[], dtype=Int64, value= 1),
 'text_label': Tensor(shape=[], dtype=String, value= 'neutral')}

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.9/site-packages/mindnlp/transformers/tokenization_utils_base.py:1526: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted, and will be then set to `False` by default. 
  warnings.warn(


In [9]:
import numpy as np
from mindnlp.dataset import BaseMapFunction
from threading import Lock
lock = Lock()

class MapFunc(BaseMapFunction):
    def __call__(self, sentence, label, text_label):
        lock.acquire()
        model_inputs = tokenizer(sentence, max_length=max_length, padding="max_length", truncation=True)
        labels = tokenizer(text_label, max_length=3, padding="max_length", truncation=True)
        lock.release()
        labels = labels['input_ids']
        labels = np.where(np.equal(labels, tokenizer.pad_token_id), -100, labels)
        return model_inputs['input_ids'], model_inputs['attention_mask'], labels


def get_dataset(dataset, tokenizer, shuffle=True):
    input_colums=['sentence', 'label', 'text_label']
    output_columns=['input_ids', 'attention_mask', 'labels']
    dataset = dataset.map(MapFunc(input_colums, output_columns),
                          input_colums, output_columns)
    if shuffle:
        dataset = dataset.shuffle(64)
    dataset = dataset.batch(batch_size)
    return dataset

train_dataset = get_dataset(train_dataset, tokenizer)
eval_dataset = get_dataset(validation_dataset, tokenizer, shuffle=False)

In [10]:
next(train_dataset.create_dict_iterator())

{'input_ids': Tensor(shape=[8, 128], dtype=Int64, value=
 [[   486,  33720,   9340 ...      0,      0,      0],
  [   259,  85336,    714 ...      0,      0,      0],
  [   486,  21511,    345 ...      0,      0,      0],
  ...
  [169774,    262,    639 ...      0,      0,      0],
  [  9981,    259,  45814 ...      0,      0,      0],
  [   486,  21982,    345 ...      0,      0,      0]]),
 'attention_mask': Tensor(shape=[8, 128], dtype=Int64, value=
 [[1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  ...
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0]]),
 'labels': Tensor(shape=[8, 3], dtype=Int64, value=
 [[59006,     1,  -100],
  [59006,     1,  -100],
  [59006,     1,  -100],
  ...
  [59006,     1,  -100],
  [18205,     1,  -100],
  [59006,     1,  -100]])}

In [11]:
from mindnlp.core import optim
# optimizer and lr scheduler
optimizer = optim.AdamW(model.trainable_params(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataset) * num_epochs),
)

In [ ]:
from mindnlp.core import value_and_grad
# training and evaluation
def forward_fn(**batch):
    outputs = model(**batch)
    loss = outputs.loss
    return loss

grad_fn = value_and_grad(forward_fn, model.trainable_params())

for epoch in range(num_epochs):
    model.set_train()
    total_loss = 0
    train_total_size = train_dataset.get_dataset_size()
    for step, batch in enumerate(tqdm(train_dataset.create_dict_iterator(), total=train_total_size)):
        optimizer.zero_grad()
        loss = grad_fn(**batch)
        optimizer.step()
        total_loss += loss.float()
        lr_scheduler.step()

    model.set_train(False)
    eval_loss = 0
    eval_preds = []
    eval_total_size = eval_dataset.get_dataset_size()
    for step, batch in enumerate(tqdm(eval_dataset.create_dict_iterator(), total=eval_total_size)):
        with mindspore._no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.float()
        eval_preds.extend(
            tokenizer.batch_decode(ops.argmax(outputs.logits, -1).asnumpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataset)
    eval_ppl = ops.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataset)
    train_ppl = ops.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

  0%|                                                                                                                       | 0/255 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:11<00:00,  2.62it/s]


epoch=0: train_ppl=Tensor(shape=[], dtype=Float32, value= 1.35495) train_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.303762) eval_ppl=Tensor(shape=[], dtype=Float32, value= 1.09358) eval_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.0894594)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:10<00:00,  2.77it/s]


epoch=1: train_ppl=Tensor(shape=[], dtype=Float32, value= 1.06017) train_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.0584264) eval_ppl=Tensor(shape=[], dtype=Float32, value= 1.03234) eval_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.0318265)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:10<00:00,  2.78it/s]

epoch=2: train_ppl=Tensor(shape=[], dtype=Float32, value= 1.03601) train_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.0353789) eval_ppl=Tensor(shape=[], dtype=Float32, value= 1.0308) eval_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.0303333)


In [13]:
# print accuracy
correct = 0
total = 0

ground_truth = []

for pred, data in zip(eval_preds, validation_dataset.create_dict_iterator(output_numpy=True)):
    true = str(data['text_label'])
    ground_truth.append(true)
    if pred.strip() == true.strip():
        correct += 1
    total += 1
accuracy = correct / total * 100
print(f"{accuracy=} % on the evaluation dataset")
print(f"{eval_preds[:10]=}")
print(f"{ground_truth[:10]=}")

accuracy=97.34513274336283 % on the evaluation dataset
eval_preds[:10]=['neutral', 'neutral', 'neutral', 'neutral', 'positive', 'positive', 'neutral', 'positive', 'positive', 'positive']
ground_truth[:10]=['neutral', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'positive', 'positive', 'positive']


In [14]:
next(eval_dataset.create_tuple_iterator())

[Tensor(shape=[8, 128], dtype=Int64, value=
 [[   486,   2733,   6396 ...      0,      0,      0],
  [   486,    259, 150106 ...      0,      0,      0],
  [   486,   5835,    259 ...      0,      0,      0],
  ...
  [  1385,    339,    259 ...      0,      0,      0],
  [   259,  18948,    776 ...      0,      0,      0],
  [   486,    259,  20147 ...      0,      0,      0]]),
 Tensor(shape=[8, 128], dtype=Int64, value=
 [[1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  ...
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0]]),
 Tensor(shape=[8, 3], dtype=Int64, value=
 [[59006,     1,  -100],
  [59006,     1,  -100],
  [59006,     1,  -100],
  ...
  [59006,     1,  -100],
  [59006,     1,  -100],
  [18205,     1,  -100]])]

In [15]:
# saving model
peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"
model.save_pretrained(peft_model_id)

In [16]:
ckpt = f"{peft_model_id}/adapter_model.ckpt"
!du -h $ckpt

9.1M	bigscience/mt0-large_LORA_SEQ_2_SEQ_LM/adapter_model.ckpt


In [17]:
from mindnlp.peft import PeftModel, PeftConfig

peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

In [18]:
model.set_train(False)
example = next(validation_dataset.create_dict_iterator(output_numpy=True))

print(example['text_label'])
inputs = tokenizer(example['text_label'], return_tensors="ms")
print(inputs)

with mindspore._no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
    print(outputs)
    print(tokenizer.batch_decode(outputs.asnumpy(), skip_special_tokens=True))

neutral
{'input_ids': Tensor(shape=[1, 2], dtype=Int64, value=
[[59006,     1]]), 'attention_mask': Tensor(shape=[1, 2], dtype=Int64, value=
[[1, 1]])}
[[    0 59006     1]]
['neutral']
